In [ ]:
%load_ext autoreload
%autoreload 2

from astropy.io import fits
from astropy.table import Table
import numpy as np
import glob

import calcos

In [ ]:
asn_filenames = glob.glob('./data/nucleus 2/*/*_asn*.fits') # asn filenames for all exposures
rawtag_filenames = glob.glob('./data/nucleus 2/*/*_rawtag*.fits') # *_rawtag.fits filenames for all exposures
percents = np.concatenate(([0], np.logspace(np.log10(0.0001), np.log10(0.05),10))) # Encircled energy percent values

In [ ]:
for file in rawtag_filenames:
    fits.setval(file, 'TWOZXTAB', value='lref$2bj2256nl_2zx_mod.fits') # point rawtags to my reference file for TWOZONE

In [ ]:
# Runs calcos for every exposure and encircled energy percent
outdir = 'percents/perc_' # location for saving calcos output
for p in percents:
    try:
        print('-------------------'+str(p)+'-------------------')
        
        # Update the reference file to have the correct UPPER_OUTER and LOWER_OUTER limits
        hdu_2zx = fits.open('./data/cal_ref/2bj2256nl_2zx_mod.fits', mode='update')
        hdu_2zx[1].data['UPPER_OUTER'] = 1 - p/2
        hdu_2zx[1].data['LOWER_OUTER'] = p/2
        hdu_2zx.close()

        # Run calcos for all the exposures
        for filename in asn_filenames:
            calcos.calcos(filename, verbosity=0, outdir=outdir+str(p))
    except:
        print("Skipping "+str(p))
        continue